# Языковые модели

Языковые модели играют важную роль в системах распознавания речи, помогая создавать более грамотные и лексически корректные тексты. В данной работе мы будем изучать нграмные языковые модели, которые позволяют довольно легко оценить вероятность и правдоподобность текста.

В нграмной языковой модели, нграм - это последовательность из n слов в тексте. Например, в предложении "по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас", биграмами будут "по-моему мы", "мы сэкономим", "сэкономим уйму" итд. Языковые модели оценивают вероятность появления последовательности слов, исходя из статистики появления каждого из нграм в обучающей выборке.

Порядком (order) нграм языковой модели называют максимальную длину нграм, которую учитывает модель. 

Практическая работа разделена на 2 части: 
1. Построение нграмой языковой модели - основная часть, 12 баллов
1. Предсказание с помощью языковой модели - дополнительная часть, 4 балла



Полезные сслыки:
* arpa формат - https://cmusphinx.github.io/wiki/arpaformat/
* обучающие материалы - https://pages.ucsd.edu/~rlevy/teaching/2015winter/lign165/lectures/lecture13/lecture13_ngrams_with_SRILM.pdf
* обучающие материалы.2 - https://cjlise.github.io/machine-learning/N-Gram-Language-Model/

In [1]:
import numpy as np
from collections import defaultdict
from typing import List, Dict, Tuple

# 1. Построение нграмной языковой модели. (12 баллов)


Вероятность текста с помощью нграмной языковой модели можно вычислить по формуле: 
$$ P(w_1, w_2, .., w_n) = {\prod{{P_{i=0}^{n}(w_i| w_{i-order}, .., w_{i-1})}}} $$

В простом виде, при обучении нграмной языковой модели, чтобы рассчитать условную вероятность каждой нграмы, используется формула, основанная на количестве появлений нграмы в обучающей выборке. Формула выглядит следующим образом:
$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i})} \over {count(w_{i-order},..., w_{i-1})}} $$

Поскольку униграмы не содержат в себе какого-дибо контекста, вероятность униграмы можно посчитать поделив кол-во этой слова на общее количество слов в обучающей выборке. 


In [2]:
# в первую очередь нам понадобится подсчитать статистику по обучающей выборке 
def count_ngrams(train_text: List[str], order=3, bos=True, eos=True) -> Dict[Tuple[str], int]:
    ngrams = defaultdict(int)
    # TODO реализуйте функцию, которая подсчитывает все 1-gram 2-gram ... order-gram ngram'ы в тексте
    
    # 
    return dict(ngrams)

In [3]:
def count_ngrams(train_text: List[str], order: int = 3, bos: bool = True, eos: bool = True) -> Dict[Tuple[str], int]:
    ngrams = defaultdict(int)
    
    for sentence in train_text:
        # разбиваем предложение на слова
        words = sentence.split()
        if bos:
            # добавляем символ начала предложения
            words = ['<s>'] + words
        if eos:
            # добавляем символ конца предложения
            words += ['</s>']

        # для каждого порядка от 1 до order
        for o in range(1, order + 1):
            # итерируемся по словам в предложении
            for i in range(len(words) - o + 1):
                # получаем n-грамму
                ngram = tuple(words[i:i + o])
                ngrams[ngram] += 1

    return dict(ngrams)

train_text = ["Костянчик купил тачку", "тачки это круто"]
ngrams = count_ngrams(train_text, order=3)
print(ngrams)

{('<s>',): 2, ('Костянчик',): 1, ('купил',): 1, ('тачку',): 1, ('</s>',): 2, ('<s>', 'Костянчик'): 1, ('Костянчик', 'купил'): 1, ('купил', 'тачку'): 1, ('тачку', '</s>'): 1, ('<s>', 'Костянчик', 'купил'): 1, ('Костянчик', 'купил', 'тачку'): 1, ('купил', 'тачку', '</s>'): 1, ('тачки',): 1, ('это',): 1, ('круто',): 1, ('<s>', 'тачки'): 1, ('тачки', 'это'): 1, ('это', 'круто'): 1, ('круто', '</s>'): 1, ('<s>', 'тачки', 'это'): 1, ('тачки', 'это', 'круто'): 1, ('это', 'круто', '</s>'): 1}


In [4]:
def test_count_ngrams():
    assert count_ngrams(['привет привет как дела'], order=1, bos=True, eos=True) == {
        ('<s>',): 1, 
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=True) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=2, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1,
        ('привет', 'привет'): 1,
        ('привет', 'как'): 1,
        ('как', 'дела'): 1
    }    
    assert count_ngrams(['привет ' * 6], order=2, bos=False, eos=False) == {
        ('привет',): 6, 
        ('привет', 'привет'): 5
    }
    result = count_ngrams(['практическое сентября',
                           'второе практическое занятие пройдет в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=5)
    assert result[('<s>',)] == 3
    assert result[('32',)] == 3
    assert result[('<s>', 'в', 'офлайне', 'в', '32')] == 1
    assert result[('офлайне', 'в', '32', '12', '</s>')] == 1
    print('Test 1a passed')
    
    
test_count_ngrams()  

Test 1a passed



Простой подход к вычислению вероятностей через количество нграм имеет существенный недостаток. Если в тексте встретится нграмма, которой не было в обучающей выборке, то вероятность всего текста будет равна нулю. 

Чтобы избежать данного недостатка, вводится специальное сглаживание - [сглаживание Лапласа](https://en.wikipedia.org/wiki/Additive_smoothing). Данная техника позволяет учитывать нграмы, не встретившиеся в обучающей выборке, и при этом не делает вероятность текста равной нулю.

Формула сглаживания Лапласа выглядит следующим образом:

$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i}) + k} \over {count(w_{i-order},..., w_{i-1}) + k*V}} $$

Здесь V - количество слов в словаре, а k - гиперпараметр, который контролирует меру сглаживания. Как правило, значение k выбирается экспериментально, чтобы найти оптимальный баланс между учетом редких нграм и сохранением вероятности для часто встречающихся нграм.


In [5]:
# функция подсчета вероятности через количество со сглаживанием Лапласа
def calculate_ngram_prob(ngram: Tuple[str], counts: Dict[Tuple[str], int], V=None, k=0) -> float:
    # подсчитывет ngram со сглаживанием Лапласа
    # TODO
    return prob

In [6]:
# функция подсчета вероятности через количество со сглаживанием Лапласа
def calculate_ngram_prob(ngram: Tuple[str], counts: Dict[Tuple[str], int], V=None, k=0) -> float:
    # количество вхождений данной n-граммы + параметр сглаживания k
    numerator = counts.get(ngram, 0) + k
    # получаем n-грамму для знаменателя (без последнего слова)
    denominator_ngram = ngram[:-1]
    # сумма всех вхождений n-грамм, начинающихся с denominator_ngram
    denominator = sum(value for key, value in counts.items() if key[:-1] == denominator_ngram)

    # если V (количество уникальных слов в словаре) не задано, вычисляем его
    if V is None:
        V = len(set([key[-1] for key in counts.keys()]))  

    # формула сглаживания Лапласа
    prob = numerator / (denominator + k * V)
    return prob

counts = count_ngrams(["Костянчик купил тачку", "тачки это круто"], order=3)
print(calculate_ngram_prob(("купил", "тачки"), counts, k=1))

0.1111111111111111


In [7]:
def test_calculate_ngram_prob():
    counts = count_ngrams(['практическое сентября',
                           'второе практическое занятие в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=4)
    assert calculate_ngram_prob(('в', 'офлайне'), counts) == 0.5
    assert calculate_ngram_prob(('в', ), counts) == 4/25
    assert calculate_ngram_prob(('в', ), counts, k=0.5) == (4+0.5)/(25+0.5*12)
    assert calculate_ngram_prob(('в', 'офлайне', 'в', '32'), counts) == 1.0
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=1) == 0.1875
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=0) == 0.0
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=1) == 0.0625
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25

    print("Test 1.b passed")
    

test_calculate_ngram_prob()  

Test 1.b passed


Основной метрикой язковых моделей является перплексия. 

Перплексия  — безразмерная величина, мера того, насколько хорошо распределение вероятностей предсказывает выборку. Низкий показатель перплексии указывает на то, что распределение вероятности хорошо предсказывает выборку.

$$ ppl = {P(w_1, w_2 ,..., w_N)^{- {1} \over {N}}} $$

In [8]:
# Языковая модель 
class NgramLM:
    def __init__(self, order=3, bos=True, eos=True, k=1, predefined_vocab=None):
        self.order = order
        self.eos = eos
        self.bos = bos
        self.k = k
        self.vocab = predefined_vocab
        self.ngrams_count = None
        
    @property
    def V(self) -> int:
        return len(self.vocab)
    
    def fit(self, train_text: List[str]) -> None:
        # TODO
        # Подсчет vocab и ngrams_count по обучающей выборке
        pass
                
    
    def predict_ngram_log_proba(self, ngram: Tuple[str]) -> float:
        # TODO 
        # считаем логарифм вероятности конкретной нграмы
        pass
           
    def predict_log_proba(self, words: List[str]) -> float:
        if self.bos:
            words = ['<s>'] + words
        if self.eos:
            words = words + ['</s>']
        logprob = 0
        # TODO 
        # применяем chain rule, чтобы посчитать логарифм вероятности всей строки
        return logprob
        
    def ppl(self, text: List[str]) -> float:
        #TODO 
        # подсчет перплексии
        # Для того, чтобы ваш код был численно стабильным, 
        #    не считайте формулу напрямую, а воспользуйтесь переходом к логарифмам вероятностей
        # 
        return perplexity

In [9]:
class NgramLM:
    def __init__(self, order=3, bos=True, eos=True, k=1, predefined_vocab=None):
        self.order = order
        self.eos = eos
        self.bos = bos
        self.k = k
        self.vocab = predefined_vocab or set()
        self.ngrams_count = defaultdict(int)

    # возвращает размер словаря
    @property
    def V(self) -> int:
        return len(self.vocab)

    # обучает модель на основе предложений в train_text
    def fit(self, train_text: List[str]) -> None:
        for text in train_text:
            tokens = text.strip().split()

            if self.bos:
                tokens = ["<s>"] + tokens

            if self.eos:
                tokens = tokens + ["</s>"]

            for n in range(0, self.order):
                for pos in range(0, len(tokens)):
                    ngram = tuple(tokens[pos:pos+n+1])

                    if len(ngram) < n + 1:
                        continue

                    self.ngrams_count[ngram] += 1

                    if len(ngram) == 1:
                        self.vocab.add(ngram[0])

    # предсказывает логарифм вероятности для заданной n-граммы
    def predict_ngram_log_proba(self, ngram: Tuple[str]) -> float:
        ngram_count = self.ngrams_count[ngram] if ngram in self.ngrams_count else 0

        n = len(ngram)

        rest_count = 0
        for ngram_text, count in self.ngrams_count.items():
            if len(ngram_text) == n and ngram_text[:-1] == ngram[:-1]:
                rest_count += count

        prob = (ngram_count + self.k) / (rest_count + (self.k * self.V))
        
        return np.log(prob)

    # предсказывает логарифм вероятности последовательности токенов
    def predict_log_proba(self, tokens: List[str], return_ngrams_count: bool = False) -> float:
        if self.bos:
            tokens = ['<s>'] + tokens
        if self.eos:
            tokens = tokens + ['</s>']
        cumulative_log_proba = 0
    
        filtered_tokens = []
        for token in tokens:
            if token.strip() != "":
                filtered_tokens.append(token)
    
        total_ngrams = 0
        for idx in range(len(filtered_tokens)):
            slice_start = max(idx + 1 - self.order, 0)
            slice_end = idx + 1
    
            current_ngram = tuple(filtered_tokens[slice_start:slice_end])
            cumulative_log_proba += self.predict_ngram_log_proba(current_ngram)
            
            total_ngrams += 1
    
        return (cumulative_log_proba, total_ngrams) if return_ngrams_count else cumulative_log_proba

    # вычисляет перплексию на тестовых предложениях
    def ppl(self, test_sentences: List[str]) -> float:
        aggregated_log_prob = 0
        total_ngrams = 0

        for test_sentence in test_sentences:
            test_words = test_sentence.strip().split()
            sentence_log_prob, ngram_count = self.predict_log_proba(test_words, return_ngrams_count=True)
            aggregated_log_prob += sentence_log_prob
            total_ngrams += ngram_count

        return np.exp(-aggregated_log_prob / total_ngrams)

In [10]:
def test_lm():
    train_data = ["по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
                  "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
                  "мы сэкономим уйму времени если я сейчас сойду с ума по-моему"]
    global lm
    lm = NgramLM(order=2)
    lm.fit(train_data)
    assert lm.V == 14
    assert np.isclose(lm.predict_log_proba(['мы']), lm.predict_log_proba(["если"]))
    assert lm.predict_log_proba(["по-моему"]) > lm.predict_log_proba(["если"]) 
    
    gt = ((3+1)/(41 + 14) * 1/(3+14))**(-1/2)
    ppl = lm.ppl([''])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    gt = ((3+1)/(41 + 14) * 1/(3+14) * 1/(14)) ** (-1/3)
    ppl = lm.ppl(['ЧТО'])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    test_data = ["по-моему если я прямо сейчас сойду с ума мы сэкономим уйму времени"]
    ppl = lm.ppl(test_data)
    assert round(ppl, 2) == 7.33, f"{ppl}"
test_lm()

# 2. Предсказания с помощью языковой модели (4 балла)

In [11]:
def predict_next_word(lm: NgramLM, prefix: List[str], topk=4):
    # TODO реализуйте функцию, которая предсказывает продолжение фразы. 
    # верните topk наиболее вероятных продолжений фразы prefix 
    return word
    

Обучите языковую модель на всем тексте из этой лабораторной работы (не забудьте заранее нормализовать текст).

Что предскажет модель по префиксам `по-моему`, `сэкономим`, `нграм` и `вероятности`? 

In [12]:
#Your code here     